models

In [174]:
''''
models.py (dataclasses, Order, exceptions)

----------------------------------------------------------------------------
Mutable Order Management
Implement an Order class with mutable attributes: symbol, quantity, price, and status.
Demonstrate in a unit test that you can update Order.status but not MarketDataPoint.price.

class OrderError(Exception)
class MarketDataPoint
class Order
class TestOrderAndMarketData(unittest.TestCase)
class MarketDataContainer:

----------------------------------------------------------------------------
Exception Handling

Define custom exceptions:

class OrderError(Exception): pass
class ExecutionError(Exception): pass
Raise OrderError for invalid orders (e.g., zero or negative quantity).
In the execution engine, simulate occasional failures and raise ExecutionError; catch and log these errors to continue processing.
'''

import unittest
from dataclasses import FrozenInstanceError
from dataclasses import dataclass
import datetime as datetime
# Exception Handling
class OrderError(Exception): 
    def __init__(self,error):
        super().__init__("This order is not valid")
class ExecutionError(Exception): pass


# define frozen data class
#immutable
@dataclass(frozen = True)
class MarketDataPoint:
    timestamp : datetime.datetime
    symbol : str
    price: float


#mutable
class Order:
    def __init__(self,symbol:str,quantity:int ,price:float,side:str,status:str):
        self.symbol = symbol
        self.quantity = quantity # validate via setter
        self.price = price # validate via setter
        self.side = side
        self.status = status
        timestamp: datetime

    @property
    def price(self):
        return self.price
    
    @price.setter
    def price(self,value):
        if value < 0:
            raise OrderError("Price cannot be negative")
        self.price = value

    @property
    def quantity(self):
        return self.quantity
    @quantity.setter
    def quantity(self, v: int):
        if v <= 0:
            raise OrderError("quantity must be > 0")
        self._quantity = int(v)

    @property
    def value(self):
        return self.price* self.quantity
    
    def __str__(self):
        return f"{self.quantity} shares of {self.symbol} at ${self.price:.2f}"
    
    def __repr__(self):
        return f"Order('{self.symbol}',{self.quantity},{self.price})"



# unit test
'''
(maybe deleted later)
'''
class TestOrderAndMarketData(unittest.TestCase):
    def test_mutable_order(self):
        order = Order("AAPL", 10, 150.0,"bid","NEW")
        self.assertEqual(order.status, "NEW")
        order.status = "FILLED"
        self.assertEqual(order.status, "FILLED")

    def test_immutable_marketdatapoint(self):
        point = MarketDataPoint(datetime.datetime.now(), "AAPL", 150.0)
        with self.assertRaises(FrozenInstanceError):
            point.price = 200.0

class MarketDataContainer:
    def __init__(self):
        self.buffer = []
        self.all_positions = {}
        self.trade_signal = []

    def buffer_data(self, data_point):
        self.buffer.append(data_point)

    def add_trade_signal(self, action, symbol, quantity, price):
        self.trade_signal.append((action, symbol, quantity, price))

    def calculate_average_price(self, position):
        if position["total_shares"] > 0:
            position["average_price"] = position["total_cost"] / position["total_shares"]
        else:
            position["average_price"] = 0.0

    def update_position(self, order):
        symbol = order['symbol']
        side = str(order['side']).upper()
        quantity = int(order['quantity'])
        price = float(order['price'])

        if symbol not in self.all_positions:
            self.all_positions[symbol] = {
                "total_shares": 0,
                "total_cost": 0.0,
                "average_price": 0.0
            }
        position = self.all_positions[symbol]

        if side == "BUY":
            position["total_shares"] += quantity
            position["total_cost"] += price * quantity
            self.calculate_average_price(position)

        elif side == "SELL":
            sell_quantity = min(quantity, position["total_shares"])
            if quantity > position["total_shares"]:
                print(f"CANNOT OVERSELL! Currently have shares: {position['total_shares']}. But wanted to sell {quantity}. PROCEED with selling {sell_quantity} shares.")

            position["total_shares"] -= sell_quantity
            position["total_cost"] -= position["average_price"] * sell_quantity
            if position["total_shares"] <= 0:
                position["total_cost"] = 0.0
                position["average_price"] = 0.0
            self.calculate_average_price(position)

    def fetch_current_position(self, symbol):
        return self.all_positions.get(symbol, {
            "total_shares": 0,
            "total_cost": 0.0,
            "average_price": 0.0
        })

data loader

In [173]:
"""
Data Ingestion & Immutable Types

Read market_data.csv (columns: timestamp, symbol, price) using the built-in csv module.

Define a frozen dataclass MarketDataPoint with attributes timestamp (datetime), symbol (str), and price (float).

Parse each row into a MarketDataPoint and collect them in a list.
"""



'''
# read data
def loadfile(csv):
    #download
    df = pd.read_csv("market_data.csv")   

    # parse row
    marketList = []       
    for i in range(len(df)):
        marketList.append(MarketDataPoint(df.iloc[i][0],df.iloc[i][1],df.iloc[i][2]))

    return marketList
'''

class MarketDataReader:
    def __init__(self, csvfile):
        self.csvfile = csvfile
        self.data_list = []

    def read_data(self):
        try:
            with open(self.csvfile, newline='') as csvfile:
                reader = csv.DictReader(csvfile)

                for row in reader:
                    try:
                        market_data_point = MarketDataPoint(
                            timestamp=datetime.datetime.fromisoformat(row["timestamp"]),
                            symbol=row["symbol"],
                            price=float(row["price"])
                        )
                        self.data_list.append(market_data_point)

                    except Exception as error_in_row:
                        print(f"There is error in row {row}: {error_in_row}")

        except Exception as error_in_csv:
            print(f"There is error reading CSV file: {error_in_csv}")
    
    def fetch_data(self):
        return self.data_list



model

In [152]:
''''
models.py (dataclasses, Order, exceptions)

----------------------------------------------------------------------------
Mutable Order Management
Implement an Order class with mutable attributes: symbol, quantity, price, and status.
Demonstrate in a unit test that you can update Order.status but not MarketDataPoint.price.

class OrderError(Exception)
class MarketDataPoint
class Order
class TestOrderAndMarketData(unittest.TestCase)
class MarketDataContainer:

----------------------------------------------------------------------------
Exception Handling

Define custom exceptions:

class OrderError(Exception): pass
class ExecutionError(Exception): pass
Raise OrderError for invalid orders (e.g., zero or negative quantity).
In the execution engine, simulate occasional failures and raise ExecutionError; catch and log these errors to continue processing.
'''

import unittest
from dataclasses import FrozenInstanceError
from dataclasses import dataclass
import datetime as datetime
# Exception Handling
class OrderError(Exception): 
    def __init__(self,error):
        super().__init__("This order is not valid")
class ExecutionError(Exception): pass


# define frozen data class
#immutable
@dataclass(frozen = True)
class MarketDataPoint:
    timestamp : datetime.datetime
    symbol : str
    price: float


#mutable
class Order:
    def __init__(self,symbol:str,quantity:int ,price:float,side:str,status:str):
        self.symbol = symbol
        self.quantity = quantity # validate via setter
        self.price = price # validate via setter
        self.side = side
        self.status = status

    @property
    def price(self):
        return self.price
    
    @price.setter
    def price(self,value):
        if value < 0:
            raise OrderError("Price cannot be negative")
        self.price = value

    @property
    def quantity(self):
        return self.quantity
    @quantity.setter
    def quantity(self, v: int):
        if v <= 0:
            raise OrderError("quantity must be > 0")
        self._quantity = int(v)

    @property
    def value(self):
        return self.price* self.quantity
    
    def __str__(self):
        return f"{self.quantity} shares of {self.symbol} at ${self.price:.2f}"
    
    def __repr__(self):
        return f"Order('{self.symbol}',{self.quantity},{self.price})"



# unit test
'''
(maybe deleted later)
'''
class TestOrderAndMarketData(unittest.TestCase):
    def test_mutable_order(self):
        order = Order("AAPL", 10, 150.0,"bid","NEW")
        self.assertEqual(order.status, "NEW")
        order.status = "FILLED"
        self.assertEqual(order.status, "FILLED")

    def test_immutable_marketdatapoint(self):
        point = MarketDataPoint(datetime.datetime.now(), "AAPL", 150.0)
        with self.assertRaises(FrozenInstanceError):
            point.price = 200.0


class MarketDataContainer:
    def __init__(self):
        self.buffer = []
        self.all_positions = {}
        self.trade_signal = []

    def buffer_data(self, data_point):
        self.buffer.append(data_point)

    def add_trade_signal(self, action, symbol, quantity, price):
        self.trade_signal.append((action, symbol, quantity, price))

    def calculate_average_price(self, position):
        if position["total_shares"] > 0:
            position["average_price"] = position["total_cost"] / position["total_shares"]
        else:
            position["average_price"] = 0.0

    def last(self):
        return self.buffer[-1] if self.buffer else None

    def recent(self, n):
        return self.buffer[-n:] if n > 0 else []

    def __len__(self):
        return len(self.buffer)

    def __iter__(self):
        return iter(self.buffer)
    
    def update_position(self, order):
        symbol = order['symbol']
        side = str(order['side']).upper()
        quantity = int(order['quantity'])
        price = float(order['price'])

        if symbol not in self.all_positions:
            self.all_positions[symbol] = {
                "total_shares": 0,
                "total_cost": 0.0,
                "average_price": 0.0
            }
        position = self.all_positions[symbol]

        if side == "BUY":
            position["total_shares"] += quantity
            position["total_cost"] += price * quantity
            self.calculate_average_price(position)

        elif side == "SELL":
            sell_quantity = min(quantity, position["total_shares"])
            if quantity > position["total_shares"]:
                print(f"CANNOT OVERSELL! Currently have shares: {position['total_shares']}. But wanted to sell {quantity}. PROCEED with selling {sell_quantity} shares.")

            position["total_shares"] -= sell_quantity
            position["total_cost"] -= position["average_price"] * sell_quantity
            if position["total_shares"] <= 0:
                position["total_cost"] = 0.0
                position["average_price"] = 0.0
            self.calculate_average_price(position)

    def fetch_current_position(self, symbol):
        return self.all_positions.get(symbol, {
            "total_shares": 0,
            "total_cost": 0.0,
            "average_price": 0.0
        })
    
class Portfolio:
    """Simple long-only position tracker per symbol."""
    def __init__(self) -> None:
        self.cash: float = 0.0
        self.positions: Dict[str, Dict[str, float]] = {}

    def _ensure(self, symbol: str):
        if symbol not in self.positions:
            self.positions[symbol] = {
                "total_shares": 0.0,
                "total_cost": 0.0,
                "average_price": 0.0,
                "realized_pnl": 0.0,
            }
        return self.positions[symbol]

    def apply_fill(self, order: Order) -> None:
        px = order.price
        qty = float(order.quantity)
        pos = self._ensure(order.symbol)
        if order.side == "BUY":
            pos["total_shares"] += qty
            pos["total_cost"] += px * qty
            pos["average_price"] = 0.0 if pos["total_shares"] == 0 else pos["total_cost"] / pos["total_shares"]
        elif order.side == "SELL":
            if qty > pos["total_shares"]:
                # allow partial sell of what's available
                qty = pos["total_shares"]
            # realized PnL on sold shares
            pos["realized_pnl"] += (px - pos["average_price"]) * qty
            pos["total_shares"] -= qty
            pos["total_cost"] = pos["average_price"] * pos["total_shares"]
            pos["average_price"] = 0.0 if pos["total_shares"] == 0 else pos["total_cost"] / pos["total_shares"]
        else:
            raise OrderError(f"Unsupported side {order.side}")

    def snapshot(self) -> Dict[str, Dict[str, float]]:
        return {k: v.copy() for k, v in self.positions.items()}


strategy

In [184]:

from __future__ import annotations
from abc import ABC, abstractmethod
from collections import deque
from typing import Dict, List, Tuple, Optional

# signal tuple: (side, symbol, qty)
Signal = Tuple[str, str, int]

class Strategy(ABC):
    @abstractmethod
    def generate_signals(self, tick, positions: Optional[Dict[str, Dict[str, float]]] = None) -> List[Signal]:
        ...

class MovingAverageCrossover(Strategy):
    def __init__(self, symbol: str, short_window: int = 5, long_window: int = 20, trade_qty: int = 1):
        if not (1 <= short_window < long_window):
            raise ValueError("Require 1 <= short_window < long_window")
        self.symbol = symbol
        self._short_w = short_window
        self._long_w = long_window
        self._prices = deque(maxlen=long_window)
        self._prev_diff = None
        self._qty = trade_qty

    def generate_signals(self, tick, positions=None):
        if tick.symbol != self.symbol:
            return []
        self._prices.append(tick.price)
        if len(self._prices) < self._long_w:
            return []
        short_ma = sum(list(self._prices)[-self._short_w:]) / self._short_w
        long_ma = sum(self._prices) / self._long_w
        diff = short_ma - long_ma

        signals: List[Signal] = []
        if self._prev_diff is not None:
            # Cross up
            if self._prev_diff <= 0 and diff > 0:
                signals.append(("BUY", tick.symbol, self._qty))
            # Cross down
            elif self._prev_diff >= 0 and diff < 0:
                # sell only up to current shares
                current = 0
                if positions and tick.symbol in positions:
                    current = int(positions[tick.symbol].get("total_shares", 0))
                qty = min(current, self._qty) if current > 0 else 0
                if qty > 0:
                    signals.append(("SELL", tick.symbol, qty))
        self._prev_diff = diff
        return signals

class Momentum(Strategy):
    def __init__(self, symbol: str, lookback: int = 10, threshold_pct: float = 0.01, trade_qty: int = 1):
        if lookback < 1:
            raise ValueError("lookback must be >= 1")
        if threshold_pct < 0:
            raise ValueError("threshold_pct must be >= 0")
        self.symbol = symbol
        self._window = lookback
        self._th = threshold_pct
        self._prices = deque(maxlen=lookback)
        self._qty = trade_qty

    def generate_signals(self, tick, positions=None):
        if tick.symbol != self.symbol:
            return []
        self._prices.append(tick.price)
        if len(self._prices) < self._window:
            return []
        now = self._prices[-1]
        past = self._prices[0]
        if past <= 0:
            return []
        change = (now - past) / past
        signals: List[Signal] = []
        if change >= self._th:
            signals.append(("BUY", tick.symbol, self._qty))
        elif change <= -self._th:
            current = 0
            if positions and tick.symbol in positions:
                current = int(positions[tick.symbol].get("total_shares", 0))
            qty = min(current, self._qty) if current > 0 else 0
            if qty > 0:
                signals.append(("SELL", tick.symbol, qty))
        
        return signals


In [129]:

# NOT USING IT
def generateSignals(marketdataPT,positions):
    '''
    genenrate siganls by two strategies
    '''
    #setup
    mac = MovingAverageCrossover()
    mom = Momentum()
    print(marketdataPT)
    MA_signals = mac.generate_signals(tick = marketdataPT, positions = positions)
    mont_signals = mom.generate_signals(tick = marketdataPT, positions = positions)
    return MA_signals,mont_signals

def Makeorder(signal):
    '''
    turn signals into order and validate

    signal format is ("SELL", tick.symbol, self._qty, tick.price)
    '''

    action, symbol, qty, price = signal
    if action not in ("BUY", "SELL"):
        raise OrderError("action must be BUY or SELL")
    if qty <= 0:
        raise OrderError("quantity must be > 0")
    if price <= 0:
        raise OrderError("price must be > 0")
    if action == "SELL":
        held = int(positions.get(symbol, {}).get("total_shares", 0))
        if held < qty:
            raise OrderError(f"insufficient shares to sell (have {held}, need {qty})")

def Makeorders(signals, positions):
    """
    Convert a list of signals into order dicts.
    signal format: ("BUY"/"SELL", symbol, qty, price)
    """
    orders = []
    for sig in signals:
        if len(sig) != 4:
            raise OrderError(f"Signal must be (side, symbol, qty, price), got {sig}")
        side, symbol, qty, price = sig
        side = str(side).upper()

        if side not in ("BUY", "SELL"):
            raise OrderError("action must be BUY or SELL")
        if qty <= 0:
            raise OrderError("quantity must be > 0")
        if price <= 0:
            raise OrderError("price must be > 0")
        if side == "SELL":
            held = int(positions.get(symbol, {}).get("total_shares", 0))
            if held < qty:
                # either raise, or downgrade to partial sell; here we raise to be explicit
                raise OrderError(f"insufficient shares to sell (have {held}, need {qty})")

        orders.append({
            "side": side,
            "symbol": symbol,
            "quantity": int(qty),
            "price": float(price),
        })
    return orders

In [206]:
listtest = []
if listtest:
    print(1)

In [225]:
from typing import List, Dict, Iterable, Tuple
class BackTestengine:
    """Drives the backtest: ticks -> signals -> orders -> fills -> portfolio updates."""
    def __init__(self, strategies: Iterable[Strategy]):
        self.strategies = list(strategies)
        self.portfolio = Portfolio()
        self.order_log: List[Order] = []
        self.error_log: List[str] = []

    def _create_orders(self, signals: List[Tuple[str,str,int]], tick: MarketDataPoint):
        orders: List[Order] = []
        for side, symbol, qty in signals:
            o = Order(side=side, symbol=symbol, quantity=int(qty), price=float(tick.price), timestamp=tick.timestamp)
            o.validate()
            orders.append(o)
        return orders

    def _execute(self, order: Order) -> None:
        # simulate flaky execution 3% of the time
        if random.random() < 0.03:
            raise ExecutionError("Simulated venue outage")
        # in a toy engine, all valid orders fill at provided price
        order.status = "FILLED"
        self.portfolio.apply_fill(order)

    def on_tick(self, tick: MarketDataPoint) -> None:
        positions = self.portfolio.snapshot()
        for strat in self.strategies:
            try:    
                signals = strat.generate_signals(tick, positions)

                if not signals:
                    continue
                orders = self._create_orders(signals, tick)
                
                for order in orders:
                    try:
                        self._execute(order)
                    except Exception as ex:
                        order.status = "REJECTED"
                        self.error_log.append(f"{tick.timestamp} {order.symbol} {order.side} x{order.quantity}: EXECUTION ERROR: {ex}")
                    finally:
                        self.order_log.append(order)
            except Exception as ex:
                self.error_log.append(f"{tick.timestamp} Strategy {type(strat).__name__} error: {ex}")

    def run(self, market: Iterable[MarketDataPoint]) -> None:
        for tick in sorted(market, key=lambda t: t.timestamp):
            self.on_tick(tick)

    def report(self) -> Dict:
        return {
            "positions": self.portfolio.snapshot(),
            "orders": [{
                "time": o.timestamp.isoformat(),
                "symbol": o.symbol,
                "side": o.side,
                "qty": o.quantity,
                "price": o.price,
                "status": o.status
            } for o in self.order_log],
            "errors": list(self.error_log),
        }


In [135]:
reader = MarketDataReader("market_data.csv")
reader.read_data()
data = reader.fetch_data()
data  = sorted(data, key=lambda t: t.timestamp)
allData = MarketDataContainer()
for dp in data:
    allData.buffer_data(dp)

positions = allData.all_positions




In [162]:
type(subset)

list

In [226]:
#collect symbols in the dataset
symbols = sorted({t.symbol for t in allData.buffer})

#  build one instance per strategy per symbol
strats = []
for sym in symbols:
    strats.append(MovingAverageCrossover(symbol=sym, short_window=3, long_window=8, trade_qty=2))
    strats.append(Momentum(symbol=sym, lookback=5, threshold_pct=0.01, trade_qty=1))


# run the engine over ALL ticks (strategies will ignore other symbols)
engine = BackTestengine(strats)
engine.run(sorted(allData.buffer, key=lambda t: t.timestamp))

report = engine.report()

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [211]:
report

{'positions': {},
 'orders': [],
 'errors': ["2025-09-20 14:30:22.162233 Strategy MovingAverageCrossover error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.162233 Strategy Momentum error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.174778 Strategy Momentum error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.187338 Strategy Momentum error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.199891 Strategy Momentum error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.339419 Strategy Momentum error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.351957 Strategy MovingAverageCrossover error: Order.__init__() got an unexpected keyword argument 'timestamp'",
  "2025-09-20 14:30:22.351957 Strategy Momentum error: Order.__init__() got an une